In [ ]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mi_scores = mutual_info_classif(X_train_scaled, y_train)
mi_feature_indices = np.argsort(mi_scores)[-2100:]  # Select top 2100 features
X_train_mi = X_train_scaled[:, mi_feature_indices]
X_val_mi = X_test_scaled[:, mi_feature_indices]

param_distributions = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1, 1],
    'kernel': ['rbf']
}

# Randomized search
random = RandomizedSearchCV(
    estimator=SVC(probability=True, class_weight='balanced'),
    param_distributions=param_distributions,  
    scoring='roc_auc',
    n_iter=10,  # How many random combinations to try
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

random.fit(X_train_mi, y_train)

best_svm = random.best_estimator_
print("Best Parameters:", random.best_params_)
print("Best ROC AUC (CV):", random.best_score_)

y_pred = best_svm.predict(X_val_mi)
y_prob = best_svm.predict_proba(X_val_mi)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("ROC AUC Score:", roc_auc_score(y_test, y_prob))

# --- Confusion Matrix ---
cm = confusion_matrix(y_test, y_pred)
labels = ['Normal', 'Abnormal']

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# --- Learning Curve ---
train_sizes, train_scores, val_scores = learning_curve(
    estimator=best_svm,
    X=X_train_mi,
    y=y_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    shuffle=True,
    random_state=42
)

train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label="Training Score", color="blue")
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color="blue")
plt.plot(train_sizes, val_mean, label="Validation Score", color="orange")
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color="orange")
plt.title("Learning Curve (SVM with Top 100 MI Features)")
plt.xlabel("Training Set Size")
plt.ylabel("ROC AUC Score")
plt.legend(loc="best")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

# Flip the labels: 1 = Normal, 0 = Abnormal
y_flipped = 1 - y

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_flipped, stratify=y_flipped, test_size=0.2, random_state=42
)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Mutual information for feature selection
mi_scores = mutual_info_classif(X_train_scaled, y_train)
mi_feature_indices = np.argsort(mi_scores)[-2100:]  # Select top 2100 features
X_train_mi = X_train_scaled[:, mi_feature_indices]
X_val_mi = X_test_scaled[:, mi_feature_indices]

# Define hyperparameter search space
param_distributions = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1, 1],
    'kernel': ['rbf']
}

# Randomized search
random = RandomizedSearchCV(
    estimator=SVC(probability=True, class_weight='balanced'),
    param_distributions=param_distributions,
    scoring='roc_auc',
    n_iter=10,
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

random.fit(X_train_mi, y_train)

best_svm = random.best_estimator_
print("Best Parameters:", random.best_params_)
print("Best ROC AUC (CV):", random.best_score_)

# Predict using the flipped label model
y_prob = best_svm.predict_proba(X_val_mi)[:, 1]  # Probability of being NORMAL
threshold = 0.65
y_pred_flipped = (y_prob >= threshold).astype(int)     # Predict normal if high confidence

# Flip predictions and truth back for interpretation: 1 = Abnormal, 0 = Normal
y_test_original = 1 - y_test
y_pred_original = 1 - y_pred_flipped

print("\nClassification Report:")
print(classification_report(y_test_original, y_pred_original))

print("Confusion Matrix:")
cm = confusion_matrix(y_test_original, y_pred_original)
print(cm)

print("ROC AUC Score:", roc_auc_score(y_test_original, y_prob))

# --- Confusion Matrix Plot ---
labels = ['Normal', 'Abnormal']
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (flipped model)')
plt.tight_layout()
plt.show()

# --- Learning Curve ---
train_sizes, train_scores, val_scores = learning_curve(
    estimator=best_svm,
    X=X_train_mi,
    y=y_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    shuffle=True,
    random_state=42
)

train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label="Training Score", color="blue")
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color="blue")
plt.plot(train_sizes, val_mean, label="Validation Score", color="orange")
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color="orange")
plt.title("Learning Curve (SVM trained to detect Normal ECGs)")
plt.xlabel("Training Set Size")
plt.ylabel("ROC AUC Score (normal class)")
plt.legend(loc="best")
plt.grid(True)
plt.tight_layout()
plt.show()

# ROC AUC curve

fpr, tpr, thresholds = roc_curve(y_test_original, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()
